In [276]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import statsmodels.api as sm


In [ ]:
# Financing
int_rate: 0.03 # Interest rate
loan_val: 25000 # Start-up loan

In [23]:
# Set up dictionaries for USERS

# Dictionary of odds on traits
user_odds = {
    'location': {
        'domestic': 0.667,
        'eu': 0.167,
        'international': 0.166
    },
    'payment_type': {
        'single': 0.35,
        'recurring': 0.65
    },
    'user_type': {
        'personal': 0.8,
        'business': 0.2
    },
    'whale': {
        'yes': 0.01,
        'no': 0.99
    }
}


# Set up dictionaries for EXPENSES

# Recurring costs
variable_costs = { # £ + % instance
    'payment_gateway': {
        'credit_cards': {
            'flat_charge': 0.2, # £0.20
            'fx': 0.02,
            'personal': {
                'domestic': 0.015,
                'eu': 0.025,
                'international': 0.0325
            },
            'business':{
                'domestic': 0.019,
                'eu': 0.025,
                'international': 0.0325
            },
        },
        'recurring_trans': 0.007 # %
    },
}

fixed_costs = { # £ annual
    'website': {
        'hosting': 240,
        'domain': 15,
        'ssl': 50,
        'maintenance': 2000
    },
    'testing': {
        'low': 300,
        'high': 3000
    },
    'business_intel': {
        'low': 120,
        'high': 2400
    },
    'crm': {
        'low': 120,
        'high': 3600
    },
    'banking': {
        'low': 120,
        'high': 360
    },
    'accounting': {
        'low': 300,
        'high': 500
    },
    'rent': {
        'strath': 0,
        'low': 8000,
        'high': 12500
    },
    'electricity': {
        'strath': 0,
        'low': 5400,
        'high': 14400
    },
    'insurance': {
        'strath': 0,
        'low': 120,
        'high': 600
    },
}

# Non-recurring costs

setup_costs = {
    'platform': {
        'low': 5000,
        'high': 10000
    },
    'branding': {
        'low': 2000,
        'high': 12500
    }
}

    

In [31]:
# User-trait df

# Function to generate users
def genUsers(trait_probs, size):
    categories = list(trait_probs.keys())
    probabilities = list(trait_probs.values())
    return np.random.choice(categories, size = size, p = probabilities)

# Number of users
num_users = 1000

# Generate a dictionary
user_data = {}
for trait, probabilities in user_odds.items():
    user_data[trait] = genUsers(probabilities, num_users)

#*********************
# Create DataFrame
user_df = pd.DataFrame(user_data)

# Generate payment amounts with a mean of £10 (clipped £1 - £100)
np.random.seed(27)  # For reproducibility
payments = np.random.normal(loc = 10, scale = 5, size = num_users)
payments = np.clip(payments, 1, 100)  # Ensure payments are between £1 and £100

# Adjust payments for whales
user_df['payment'] = payments
user_df.loc[user_df['whale'] == 'yes', 'payment'] *= 10 # Whales pay 10x as much

user_df['payment'] = np.round(user_df['payment'], 2) # Round payments to 2dp

#*********************
# Calculate ransaction fees
def transFee(row):
    payment = row['payment']
    payment_type = row['payment_type']
    user_type = row['user_type']
    location = row['location']
    
    # Retrieve relevant costs
    gateway = variable_costs['payment_gateway']
    credit_cards = gateway['credit_cards']
    
    if payment_type == 'recurring':
        # Recurring transactions use the recurring transaction percentage
        fee = payment * gateway['recurring_trans']
    else:
        # Single transactions use flat_charge + other percentage fees
        fee = credit_cards['flat_charge']
        fee += payment * credit_cards[user_type][location]  # User/location-based percentage
        if location != 'domestic':  # Foreign exchange fee applies only to non-domestic
            fee += payment * credit_cards['fx']
    
    return round(fee, 2)  # Return fee (2dp)

# Apply the fee calculation to each user
user_df['processing_fee'] = user_df.apply(transFee, axis=1)

#*********************
# Annualise payments & calculate total fees
def anVals(row):
    payment = row['payment']
    payment_type = row['payment_type']
    processing_fee = row['processing_fee']
    
    if payment_type == 'recurring': # Monthly payments
        annual_payment = payment * 12
        total_fees = processing_fee * 12
    else:
        # Single payment is a one-off
        annual_payment = payment
        total_fees = processing_fee
    
    # Net annualised payment after fees
    na_payment = annual_payment - total_fees
    
    return annual_payment, total_fees, na_payment

# Apply the calculation to each user
user_df[['annual_payment', 'total_fees', 'na_payment']] = user_df.apply(
    lambda row: pd.Series(anVals(row)), axis=1
)

#********************
# Display df
user_df


,location,payment_type,user_type,whale,payment,processing_fee,annual_payment,total_fees,na_payment
0,international,recurring,personal,no,16.43,0.12,197.16,1.44,195.72
1,domestic,recurring,personal,no,8.48,0.06,101.76,0.72,101.04
2,international,recurring,business,no,13.10,0.09,157.20,1.08,156.12
3,domestic,single,business,no,11.98,0.43,11.98,0.43,11.55
4,domestic,recurring,personal,no,11.12,0.08,133.44,0.96,132.48
...,...,...,...,...,...,...,...,...,...
995,domestic,recurring,personal,no,3.24,0.02,38.88,0.24,38.64
996,domestic,recurring,personal,no,14.71,0.10,176.52,1.20,175.32
997,domestic,recurring,personal,no,9.57,0.07,114.84,0.84,114.00
998,international,single,personal,no,3.70,0.39,3.70,0.39,3.31


In [57]:
# User base growth simulation

# Ranges for each parameter
initial_users_range = [100, 200, 300, 400, 500]
initial_grate_range = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5]
acc_rate_range = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]

# List to collect the results
all_results = []

# Loop through all combinations of parameters
for initial_users, initial_grate, acc_rate in itertools.product(initial_users_range, initial_grate_range, acc_rate_range):
    user_growth = []
    annual_grates = []
    
    # Simulate userbase growth for combo
    for t in range(6):  # 5 years + year 0
        growth_rate = initial_grate + (acc_rate * t)
        annual_grates.append(growth_rate)
        
        # Calculate the number of users at year t
        users_t = initial_users * (1 + growth_rate)**t
        user_growth.append(users_t)
    
    # Store the results for this scenario
    scenario_data = {
        'initial_users': initial_users,
        'initial_grate': initial_grate,
        'acc_rate': acc_rate,
        'user_growth': user_growth
    }
    
    all_results.append(scenario_data)

# Convert the results to a DataFrame
scenario_analysis_df = pd.DataFrame(all_results)

# Expand the user growth list into individual columns for each year
expanded_df = scenario_analysis_df['user_growth'].apply(pd.Series)
expanded_df.columns = [f'Year_{i}' for i in range(6)]

# Merge with the parameter columns
final_df = pd.concat([scenario_analysis_df[['initial_users', 'initial_grate', 'acc_rate']], expanded_df], axis=1)

# Round the user growth values to the nearest integer
final_df[final_df.columns[3:]] = final_df[final_df.columns[3:]].round(0)

# Display the final DataFrame
final_df

,initial_users,initial_grate,acc_rate,Year_0,Year_1,Year_2,Year_3,Year_4,Year_5
0,100,0.05,0.01,100.0,106.0,114.0,126.0,141.0,161.0
1,100,0.05,0.05,100.0,110.0,132.0,173.0,244.0,371.0
2,100,0.05,0.10,100.0,115.0,156.0,246.0,442.0,895.0
3,100,0.05,0.15,100.0,120.0,182.0,338.0,741.0,1890.0
4,100,0.05,0.20,100.0,125.0,210.0,449.0,1171.0,3621.0
...,...,...,...,...,...,...,...,...,...
265,500,0.50,0.05,500.0,775.0,1280.0,2246.0,4176.0,8207.0
266,500,0.50,0.10,500.0,800.0,1445.0,2916.0,6516.0,16000.0
267,500,0.50,0.15,500.0,825.0,1620.0,3707.0,9724.0,28833.0
268,500,0.50,0.20,500.0,850.0,1805.0,4631.0,13992.0,48828.0


In [115]:
# Simulate user payments & fees across multiple growth conditions

#*******************************
# Set up dictionaries

# Dictionary of odds on traits
user_odds = {
    'location': {
        'domestic': 0.667,
        'eu': 0.167,
        'international': 0.166
    },
    'payment_type': {
        'single': 0.35,
        'recurring': 0.65
    },
    'user_type': {
        'personal': 0.8,
        'business': 0.2
    },
    'whale': {
        'yes': 0.01,
        'no': 0.99
    }
}

# Recurring costs
variable_costs = { # £ + % instance
    'payment_gateway': {
        'credit_cards': {
            'flat_charge': 0.2, # £0.20
            'fx': 0.02,
            'personal': {
                'domestic': 0.015,
                'eu': 0.025,
                'international': 0.0325
            },
            'business':{
                'domestic': 0.019,
                'eu': 0.025,
                'international': 0.0325
            },
        },
        'recurring_trans': 0.007 # %
    },
}

#**********************************
# Define parameter ranges
initial_users_range = [100, 200, 300, 400, 500]
initial_grate_range = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5]
acc_rate_range = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
trans_cut_range = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

#**************************************
# Function to generate user traits based on the given probabilities
def genUsers(n_users, odds):
    users = []
    for _ in range(n_users):
        user = {
            'location': np.random.choice(list(odds['location'].keys()), p=list(odds['location'].values())),
            'payment_type': np.random.choice(list(odds['payment_type'].keys()), p=list(odds['payment_type'].values())),
            'user_type': np.random.choice(list(odds['user_type'].keys()), p=list(odds['user_type'].values())),
            'whale': np.random.choice(list(odds['whale'].keys()), p=list(odds['whale'].values()))
        }
        users.append(user)
    return pd.DataFrame(users)

# Generate payment amounts with a mean of £10 (clipped £1 - £100)
def payAmount():
    payments = np.random.normal(loc = 10, scale = 5)
    payments = np.clip(payments, 1, 100)  # Ensure payments are between £1 and £100
    return payments

#*************************************
# Prepare a list to collect the results
all_results = []

# Loop through all combinations of initial_users, initial_grate, acc_rate, and trans_cut
for initial_users, initial_grate, acc_rate, trans_cut in itertools.product(initial_users_range, initial_grate_range, acc_rate_range, trans_cut_range):
    user_growth = []
    annual_grates = []
    total_payments = []
    transaction_fees = []
    net_payment = []
    net_cut = []
    
    # Simulate userbase growth for this combination
    for t in range(6):  # 5 years + year 0
        growth_rate = initial_grate + (acc_rate * t)
        annual_grates.append(growth_rate)
        
        # Calculate the number of users at year t
        users_t = initial_users * (1 + growth_rate)**t
        user_growth.append(users_t)
        
        # Generate user traits for the current year based on the number of users
        user_traits = genUsers(int(users_t), user_odds)
        
        #*******************************
       
        # Initialise counters for total payments and total fees
        total_payment_year = 0
        total_fees_year = 0
        
        # Calculate transaction fees and total payments for this year
        for _, user in user_traits.iterrows():
            payment_amount = payAmount()  # Generate payments mean £10, range 1 - 100
            payment_type = user['payment_type']
            location = user['location']
            user_type = user['user_type']
            whale = user['whale']

            # Account for 'whales'
            if whale == 'yes':
                payment_amount *= 10 # Whales pay 10x
            
            # For recurring payment, apply recurring transaction fee
            if payment_type == 'recurring':
                fee_percentage = variable_costs['payment_gateway']['recurring_trans']
                flat_fee = 0
            
            else:
                fee_percentage = 0
                flat_fee = variable_costs['payment_gateway']['credit_cards']['flat_charge']
            
            # Adjust for location
            if payment_type == 'recurring':
                payment_amount += 12 * payment_amount
                total_fees_year += 12 * (payment_amount * fee_percentage)
            else:
                if location == 'domestic':
                    
                    if user_type == 'business':
                        total_fees_year += flat_fee \
                        + (payment_amount * (variable_costs ['payment_gateway']['credit_cards']['business']['domestic']))
                    
                    else:
                        total_fees_year += flat_fee \
                        + (payment_amount * (variable_costs ['payment_gateway']['credit_cards']['personal']['domestic']))
                
                elif location == 'eu':
                    total_fees_year += flat_fee \
                    + (payment_amount * (variable_costs ['payment_gateway']['credit_cards']['personal']['eu'])) \
                    + (payment_amount * (variable_costs['payment_gateway']['credit_cards']['fx']))
                
                else:
                    total_fees_year += flat_fee \
                    + (payment_amount * (variable_costs['payment_gateway']['credit_cards']['personal']['international'])) \
                    + (payment_amount * (variable_costs['payment_gateway']['credit_cards']['fx']))
                    
            
            # Track total payments & cut for the year
            total_payment_year += payment_amount

            
        
        # Store the total payments and fees for this year
        total_payments.append(total_payment_year)
        transaction_fees.append(total_fees_year)
        net_payment.append(total_payment_year - total_fees_year)
        net_cut.append((total_payment_year - total_fees_year) * trans_cut)

    #************************************
    # Store the results for this scenario
    scenario_data = {
        'initial_users': initial_users,
        'initial_grate': initial_grate,
        'acc_rate': acc_rate,
        'trans_cut': trans_cut,  # Add trans_cut for clarity in results
        'user_growth': user_growth,
        'total_payments': total_payments,
        'transaction_fees': transaction_fees,
        'net_payment': net_payment,
        'net_cut': net_cut
    }
    
    all_results.append(scenario_data)

#********************************
# Convert the results to a DataFrame
scenario_analysis_df = pd.DataFrame(all_results)

scenario_analysis_df

,initial_users,initial_grate,acc_rate,trans_cut,user_growth,total_payments,transaction_fees,net_payment,net_cut
0,100,0.05,0.01,0.00,"[100.0, 106.0, 114.49000000000001, 125.9712000...","[7970.898299495073, 10210.74932479899, 11173.3...","[654.9090141952299, 845.0118680619971, 918.017...","[7315.989285299843, 9365.737456736992, 10255.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,100,0.05,0.01,0.05,"[100.0, 106.0, 114.49000000000001, 125.9712000...","[10641.806714101005, 9766.085684087084, 9670.0...","[883.6172910684068, 806.9637244600971, 792.050...","[9758.189423032598, 8959.121959626988, 8878.01...","[487.90947115162993, 447.9560979813494, 443.90..."
2,100,0.05,0.01,0.10,"[100.0, 106.0, 114.49000000000001, 125.9712000...","[8098.819182762468, 8851.892663549144, 11033.9...","[668.0597456332671, 731.0629521005552, 909.521...","[7430.759437129201, 8120.8297114485895, 10124....","[743.0759437129201, 812.082971144859, 1012.438..."
3,100,0.05,0.01,0.15,"[100.0, 106.0, 114.49000000000001, 125.9712000...","[10640.111643416221, 10777.957147341715, 12202...","[857.7536006154608, 888.6102081158523, 1011.21...","[9782.35804280076, 9889.346939225863, 11190.95...","[1467.3537064201141, 1483.4020408838794, 1678...."
4,100,0.05,0.01,0.20,"[100.0, 106.0, 114.49000000000001, 125.9712000...","[8883.70701805245, 9546.420896388261, 9896.936...","[723.6995403755843, 780.6394151666314, 812.623...","[8160.007477676865, 8765.78148122163, 9084.312...","[1632.0014955353731, 1753.1562962443259, 1816...."
...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,0.10,"[500.0, 875.0, 2000.0, 5695.3125, 19531.25, 78...","[50048.355756979785, 85557.53040023234, 185618...","[4139.818859129027, 7054.581781645607, 15265.7...","[45908.53689785076, 78502.94861858673, 170353....","[4590.853689785076, 7850.294861858673, 17035.3..."
1886,500,0.50,0.25,0.15,"[500.0, 875.0, 2000.0, 5695.3125, 19531.25, 78...","[53465.9109471263, 88745.7153977698, 211294.82...","[4420.664387410202, 7339.541228347886, 17452.1...","[49045.2465597161, 81406.17416942192, 193842.7...","[7356.786983957415, 12210.926125413289, 29076...."
1887,500,0.50,0.25,0.20,"[500.0, 875.0, 2000.0, 5695.3125, 19531.25, 78...","[47926.779919044064, 84899.3215492801, 198757....","[3949.1628106928965, 6999.448268961921, 16436....","[43977.617108351165, 77899.87328031818, 182320...","[8795.523421670234, 15579.974656063636, 36464...."
1888,500,0.50,0.25,0.25,"[500.0, 875.0, 2000.0, 5695.3125, 19531.25, 78...","[47442.99789543928, 94632.41947983885, 190464....","[3904.350412272962, 7820.261561491219, 15708.2...","[43538.647483166314, 86812.15791834763, 174756...","[10884.661870791579, 21703.039479586907, 43689..."


In [117]:
pd.DataFrame(scenario_analysis_df['net_cut'].tolist(), index=scenario_analysis_df.index)

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,487.909471,447.956098,443.900962,412.657543,541.387220,7.053505e+02
2,743.075944,812.082971,1012.438095,1060.486780,1113.034081,1.315140e+03
3,1467.353706,1483.402041,1678.643293,1554.714224,1730.301488,1.853455e+03
4,1632.001496,1753.156296,1816.862580,2292.301444,2382.940009,2.746918e+03
...,...,...,...,...,...,...
1885,4590.853690,7850.294862,17035.320244,50198.230888,172955.441983,6.951052e+05
1886,7356.786984,12210.926125,29076.406847,75127.233449,257908.934974,1.040097e+06
1887,8795.523422,15579.974656,36464.019990,100284.002320,347524.879631,1.391763e+06
1888,10884.661871,21703.039480,43689.084651,128860.764488,437111.552446,1.746541e+06


In [165]:


# Expand the user growth, transaction fees, and net payment lists into individual columns for each year
expanded_user_growth = pd.DataFrame(scenario_analysis_df['user_growth'].tolist(), columns=[f'User Growth (Y{i})' for i in range(6)])
expanded_total_payments = pd.DataFrame(scenario_analysis_df['total_payments'].tolist(), columns=[f'Total Payments (Y{i})' for i in range(6)])
expanded_transaction_fees = pd.DataFrame(scenario_analysis_df['transaction_fees'].tolist(), columns=[f'Total Transaction Feesf(Y{i})' for i in range(6)])
expanded_net_payment = pd.DataFrame(scenario_analysis_df['net_payment'].tolist(), columns=[f'Net Payments (Y{i})' for i in range(6)])
expanded_net_cut = pd.DataFrame(scenario_analysis_df['net_cut'].tolist(), columns=[f'Net Cut (Y{i})' for i in range(6)])

# Merge with the parameter columns
final_df = pd.concat([scenario_analysis_df[['initial_users', 'initial_grate', 'acc_rate']],
                      expanded_user_growth, 
                      expanded_total_payments, 
                      expanded_transaction_fees, expanded_net_payment,
                      expanded_net_cut], axis=1)

# Round values
final_df[final_df.columns[3:]] = final_df[final_df.columns[3:]].round(2)

final_df.rename(columns={'initial_users': 'Starting Backers', 'initial_grate': 'Starting Growth Rate', 'acc_rate': 'Growth Accelleration'}, inplace=True)

# Display the final DataFrame
final_df

,Starting Backers,Starting Growth Rate,Growth Accelleration,User Growth (Y0),User Growth (Y1),User Growth (Y2),User Growth (Y3),User Growth (Y4),User Growth (Y5),Total Payments (Y0),...,Net Payments (Y2),Net Payments (Y3),Net Payments (Y4),Net Payments (Y5),Net Cut (Y0),Net Cut (Y1),Net Cut (Y2),Net Cut (Y3),Net Cut (Y4),Net Cut (Y5)
0,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,7970.90,...,10255.34,10887.01,12854.33,15775.95,0.00,0.00,0.00,0.00,0.00,0.00
1,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10641.81,...,8878.02,8253.15,10827.74,14107.01,487.91,447.96,443.90,412.66,541.39,705.35
2,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8098.82,...,10124.38,10604.87,11130.34,13151.40,743.08,812.08,1012.44,1060.49,1113.03,1315.14
3,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10640.11,...,11190.96,10364.76,11535.34,12356.37,1467.35,1483.40,1678.64,1554.71,1730.30,1853.45
4,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8883.71,...,9084.31,11461.51,11914.70,13734.59,1632.00,1753.16,1816.86,2292.30,2382.94,2746.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,50048.36,...,170353.20,501982.31,1729554.42,6951052.33,4590.85,7850.29,17035.32,50198.23,172955.44,695105.23
1886,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,53465.91,...,193842.71,500848.22,1719392.90,6933979.50,7356.79,12210.93,29076.41,75127.23,257908.93,1040096.93
1887,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,47926.78,...,182320.10,501420.01,1737624.40,6958812.92,8795.52,15579.97,36464.02,100284.00,347524.88,1391762.58
1888,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,47443.00,...,174756.34,515443.06,1748446.21,6986165.96,10884.66,21703.04,43689.08,128860.76,437111.55,1746541.49


In [167]:
# Dictionary of FIXED COSTS

fixed_costs = { # £ annual
    'website': {
        'hosting': 240,
        'domain': 15,
        'ssl': 50,
        'maintenance': 2000
    },
    'testing': {
        'low': 300,
        'high': 3000
    },
    'business_intel': {
        'low': 120,
        'high': 2400
    },
    'crm': {
        'low': 120,
        'high': 3600
    },
    'banking': {
        'low': 120,
        'high': 360
    },
    'accounting': {
        'low': 300,
        'high': 500
    },
    'rent': {
        'strath': 0,
        'low': 8000,
        'high': 12500
    },
    'electricity': {
        'strath': 0,
        'low': 5400,
        'high': 14400
    },
    'insurance': {
        'strath': 0,
        'low': 120,
        'high': 600
    },
}

# Dictionary of SET-UP COSTS

setup_costs = {
    'platform': {
        'low': 5000,
        'high': 10000
    },
    'branding': {
        'low': 2000,
        'high': 12500
    }
}



# Loop to get low & high annual costs & fixed costs

high_key = 'high'
low_key = 'low'

# Set-up costs
high_sucosts = sum(inner_dict.get(high_key, 0) for inner_dict in setup_costs.values())
low_sucosts = sum(inner_dict.get(low_key, 0) for inner_dict in setup_costs.values())

print(f"Total of high setup costs £{high_sucosts}")
print(f"Total of low setup costs £{low_sucosts}")

# Fixed costs
high_fcosts = sum(inner_dict.get(high_key, 0) for inner_dict in fixed_costs.values())
low_fcosts = sum(inner_dict.get(low_key, 0) for inner_dict in fixed_costs.values())

print(f"Total of high fixed costs £{high_fcosts}")
print(f"Total of low fixed costs £{low_fcosts}")

# Calculate annual profit

def addProf(col):
    col_year = col[-4:]
    for i in range(len(final_df)):
        final_df[f'Profit (Conservative) {col_year}'] = final_df[col] - high_fcosts
        final_df[f'Profit (Optimistic) {col_year}'] = final_df[col] - low_fcosts

for i in range (0, 6):
    addProf(f'Net Cut (Y{i})')

final_df['Profit (Conservative) (Y0)'] = final_df['Profit (Conservative) (Y0)'] - high_sucosts
final_df['Profit (Optimistic) (Y0)'] = final_df['Profit (Optimistic) (Y0)'] - low_sucosts


final_df


Total of high setup costs £22500
Total of low setup costs £7000
Total of high fixed costs £37360
Total of low fixed costs £14480


,Starting Backers,Starting Growth Rate,Growth Accelleration,User Growth (Y0),User Growth (Y1),User Growth (Y2),User Growth (Y3),User Growth (Y4),User Growth (Y5),Total Payments (Y0),...,Profit (Conservative) (Y1),Profit (Optimistic) (Y1),Profit (Conservative) (Y2),Profit (Optimistic) (Y2),Profit (Conservative) (Y3),Profit (Optimistic) (Y3),Profit (Conservative) (Y4),Profit (Optimistic) (Y4),Profit (Conservative) (Y5),Profit (Optimistic) (Y5)
0,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,7970.90,...,-37360.00,-14480.00,-37360.00,-14480.00,-37360.00,-14480.00,-37360.00,-14480.00,-37360.00,-14480.00
1,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10641.81,...,-36912.04,-14032.04,-36916.10,-14036.10,-36947.34,-14067.34,-36818.61,-13938.61,-36654.65,-13774.65
2,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8098.82,...,-36547.92,-13667.92,-36347.56,-13467.56,-36299.51,-13419.51,-36246.97,-13366.97,-36044.86,-13164.86
3,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10640.11,...,-35876.60,-12996.60,-35681.36,-12801.36,-35805.29,-12925.29,-35629.70,-12749.70,-35506.55,-12626.55
4,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8883.71,...,-35606.84,-12726.84,-35543.14,-12663.14,-35067.70,-12187.70,-34977.06,-12097.06,-34613.08,-11733.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,50048.36,...,-29509.71,-6629.71,-20324.68,2555.32,12838.23,35718.23,135595.44,158475.44,657745.23,680625.23
1886,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,53465.91,...,-25149.07,-2269.07,-8283.59,14596.41,37767.23,60647.23,220548.93,243428.93,1002736.93,1025616.93
1887,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,47926.78,...,-21780.03,1099.97,-895.98,21984.02,62924.00,85804.00,310164.88,333044.88,1354402.58,1377282.58
1888,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,47443.00,...,-15656.96,7223.04,6329.08,29209.08,91500.76,114380.76,399751.55,422631.55,1709181.49,1732061.49


In [185]:
upper_lim = 250000
lower_lim = 50000

upper_rate = 0.25
lower_rate = 0.19

# Function to calculate marginal relief
def marginalRelief(profit):
    marginal_relief = (upper_lim - profit) * 3/200 / (upper_lim - lower_lim)
    return marginal_relief

# Function to apply tax calculation
def taxCalc(col):
    
    for i in range(len(final_df)):
        profit = final_df.loc[i, col]  # Get the profit for the current row and column

        if profit > upper_lim:
            final_df.loc[i, col] -= profit * upper_rate
        elif profit < lower_lim:
            final_df.loc[i, col] -= profit * lower_rate
        elif profit < upper_lim and profit > lower_lim:
            relief = marginalRelief(profit)
            final_df.loc[i, col] -= relief

# Apply tax calculation to each year column
for i in range(0, 6):
    taxCalc(f'Profit (Optimistic) (Y{i})')
    taxCalc(f'Profit (Conservative) (Y{i})')

final_df.round(2)

,Starting Backers,Starting Growth Rate,Growth Accelleration,User Growth (Y0),User Growth (Y1),User Growth (Y2),User Growth (Y3),User Growth (Y4),User Growth (Y5),Total Payments (Y0),...,Profit (Conservative) (Y1),Profit (Optimistic) (Y1),Profit (Conservative) (Y2),Profit (Optimistic) (Y2),Profit (Conservative) (Y3),Profit (Optimistic) (Y3),Profit (Conservative) (Y4),Profit (Optimistic) (Y4),Profit (Conservative) (Y5),Profit (Optimistic) (Y5)
0,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,7970.90,...,0.0,-7695.27,0.0,-7695.27,0.0,-7695.27,0.0,-7695.27,0.0,-7695.27
1,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10641.81,...,0.0,-7457.20,0.0,-7459.36,0.0,-7475.96,0.0,-7407.55,0.0,-7320.41
2,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8098.82,...,0.0,-7263.69,0.0,-7157.21,0.0,-7131.68,0.0,-7103.76,0.0,-6996.35
3,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10640.11,...,0.0,-6906.93,0.0,-6803.17,0.0,-6869.03,0.0,-6775.71,0.0,-6710.27
4,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8883.71,...,0.0,-6763.56,0.0,-6729.71,0.0,-6477.04,0.0,-6428.87,0.0,-6235.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,50048.36,...,0.0,-3523.30,0.0,1358.00,0.0,18982.13,0.0,158475.42,0.0,287138.77
1886,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,53465.91,...,0.0,-1205.88,0.0,7757.13,0.0,60647.19,0.0,243428.93,0.0,432682.14
1887,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,47926.78,...,0.0,584.57,0.0,11683.21,0.0,85803.96,0.0,249783.66,0.0,581041.09
1888,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18,47443.00,...,0.0,3838.62,0.0,15522.90,0.0,114380.73,0.0,237730.25,0.0,730713.44


In [272]:
# Seperate dataframes

standard_cols = ['Starting Backers', 'Starting Growth Rate', 'Growth Accelleration']

def addYears(col):
    years = []
    for i in range(0, 6):
        years.append(f'{col} (Y{i})')  # Append each year-based column name to the list
    return years  # Return the entire list of column names

growth_cols = addYears('User Growth')
np_cols = addYears('Net Payments')
cut_cols = addYears('Net Cut')
conprofit_cols = addYears('Profit (Conservative)')
opprofit_cols = addYears('Profit (Optimistic)')



# User Growth
growth_df = final_df[standard_cols + growth_cols] 

# Net Payments
payments_df = final_df[standard_cols + np_cols] 

# Net Cut
cut_df = final_df[standard_cols + cut_cols] 

# Profit
profit_df = final_df[standard_cols + opprofit_cols] 



In [254]:
growth_df

,Starting Backers,Starting Growth Rate,Growth Accelleration,User Growth (Y0),User Growth (Y1),User Growth (Y2),User Growth (Y3),User Growth (Y4),User Growth (Y5)
0,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05
1,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05
2,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05
3,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05
4,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05
...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18
1886,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18
1887,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18
1888,500,0.50,0.25,500.0,875.0,2000.00,5695.31,19531.25,78638.18


                               OLS Regression Results                               
Dep. Variable:     Profit (Optimistic) (Y3)   R-squared:                       0.319
Model:                                  OLS   Adj. R-squared:                  0.317
Method:                       Least Squares   F-statistic:                     293.9
Date:                      Thu, 05 Dec 2024   Prob (F-statistic):          1.60e-156
Time:                              22:04:00   Log-Likelihood:                -20713.
No. Observations:                      1890   AIC:                         4.143e+04
Df Residuals:                          1886   BIC:                         4.146e+04
Df Model:                                 3                                         
Covariance Type:                  nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

In [258]:
payments_df

,Starting Backers,Starting Growth Rate,Growth Accelleration,Net Payments (Y0),Net Payments (Y1),Net Payments (Y2),Net Payments (Y3),Net Payments (Y4),Net Payments (Y5)
0,100,0.05,0.01,7315.99,9365.74,10255.34,10887.01,12854.33,15775.95
1,100,0.05,0.01,9758.19,8959.12,8878.02,8253.15,10827.74,14107.01
2,100,0.05,0.01,7430.76,8120.83,10124.38,10604.87,11130.34,13151.40
3,100,0.05,0.01,9782.36,9889.35,11190.96,10364.76,11535.34,12356.37
4,100,0.05,0.01,8160.01,8765.78,9084.31,11461.51,11914.70,13734.59
...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,45908.54,78502.95,170353.20,501982.31,1729554.42,6951052.33
1886,500,0.50,0.25,49045.25,81406.17,193842.71,500848.22,1719392.90,6933979.50
1887,500,0.50,0.25,43977.62,77899.87,182320.10,501420.01,1737624.40,6958812.92
1888,500,0.50,0.25,43538.65,86812.16,174756.34,515443.06,1748446.21,6986165.96


In [260]:
cut_df

,Starting Backers,Starting Growth Rate,Growth Accelleration,Net Cut (Y0),Net Cut (Y1),Net Cut (Y2),Net Cut (Y3),Net Cut (Y4),Net Cut (Y5)
0,100,0.05,0.01,0.00,0.00,0.00,0.00,0.00,0.00
1,100,0.05,0.01,487.91,447.96,443.90,412.66,541.39,705.35
2,100,0.05,0.01,743.08,812.08,1012.44,1060.49,1113.03,1315.14
3,100,0.05,0.01,1467.35,1483.40,1678.64,1554.71,1730.30,1853.45
4,100,0.05,0.01,1632.00,1753.16,1816.86,2292.30,2382.94,2746.92
...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,4590.85,7850.29,17035.32,50198.23,172955.44,695105.23
1886,500,0.50,0.25,7356.79,12210.93,29076.41,75127.23,257908.93,1040096.93
1887,500,0.50,0.25,8795.52,15579.97,36464.02,100284.00,347524.88,1391762.58
1888,500,0.50,0.25,10884.66,21703.04,43689.08,128860.76,437111.55,1746541.49


In [274]:
profit_df

,Starting Backers,Starting Growth Rate,Growth Accelleration,Profit (Optimistic) (Y0),Profit (Optimistic) (Y1),Profit (Optimistic) (Y2),Profit (Optimistic) (Y3),Profit (Optimistic) (Y4),Profit (Optimistic) (Y5)
0,100,0.05,0.01,-11415.352680,-7695.265680,-7695.265680,-7695.265680,-7695.265680,-7695.265680
1,100,0.05,0.01,-11156.057302,-7457.201370,-7459.359020,-7475.961237,-7407.548837,-7320.413771
2,100,0.05,0.01,-11020.449502,-7263.693073,-7157.213554,-7131.677814,-7103.755904,-6996.346363
3,100,0.05,0.01,-10635.542729,-6906.926101,-6803.167560,-6869.029043,-6775.713318,-6710.266359
4,100,0.05,0.01,-10548.040968,-6763.564576,-6729.711785,-6477.043476,-6428.873663,-6235.439768
...,...,...,...,...,...,...,...,...,...
1885,500,0.50,0.25,-8975.586765,-3523.299712,1358.001816,18982.131869,158475.419407,287138.768906
1886,500,0.50,0.25,-7505.652846,-1205.876830,7757.130727,60647.187396,243428.928522,432682.142344
1887,500,0.50,0.25,-6741.052736,584.569157,11683.209573,85803.963056,249783.659968,581041.088438
1888,500,0.50,0.25,-5630.798085,3838.619601,15522.902684,114380.729486,237730.245955,730713.441094


In [284]:

# Prepare the data
X = final_df[['Starting Backers', 'Starting Growth Rate', 'Growth Accelleration', 'Net Cut (Y5)']]
y = final_df['Profit (Optimistic) (Y5)']  # Change to the target year (Y0, Y1, Y2, etc.)

# Add a constant to the model (for the intercept term)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# View the results
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     Profit (Optimistic) (Y5)   R-squared:                       0.901
Model:                                  OLS   Adj. R-squared:                  0.900
Method:                       Least Squares   F-statistic:                     4267.
Date:                      Thu, 05 Dec 2024   Prob (F-statistic):               0.00
Time:                              22:06:00   Log-Likelihood:                -22502.
No. Observations:                      1890   AIC:                         4.501e+04
Df Residuals:                          1885   BIC:                         4.504e+04
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

In [290]:
sample_values = {
    'Starting Backers': [100, 200, 300, 400, 500],
    'Starting Growth Rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5],
    'Growth Accelleration': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
}
#******************************


valid_values = {
    'Starting Backers': [100],
    'Starting Growth Rate': [0.05],
    'Growth Accelleration': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
}

filtered_df = final_df[
    final_df['Starting Backers'].isin(valid_values['Starting Backers']) &
    final_df['Starting Growth Rate'].isin(valid_values['Starting Growth Rate']) &
    final_df['Growth Accelleration'].isin(valid_values['Growth Accelleration'])
]

filtered_df

,Starting Backers,Starting Growth Rate,Growth Accelleration,User Growth (Y0),User Growth (Y1),User Growth (Y2),User Growth (Y3),User Growth (Y4),User Growth (Y5),Total Payments (Y0),...,Profit (Conservative) (Y1),Profit (Optimistic) (Y1),Profit (Conservative) (Y2),Profit (Optimistic) (Y2),Profit (Conservative) (Y3),Profit (Optimistic) (Y3),Profit (Conservative) (Y4),Profit (Optimistic) (Y4),Profit (Conservative) (Y5),Profit (Optimistic) (Y5)
0,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,7970.90,...,0.0,-7695.265680,0.0,-7695.265680,0.0,-7695.265680,0.0,-7695.265680,0.0,-7695.265680
1,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10641.81,...,0.0,-7457.201370,0.0,-7459.359020,0.0,-7475.961237,0.0,-7407.548837,0.0,-7320.413771
2,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8098.82,...,0.0,-7263.693073,0.0,-7157.213554,0.0,-7131.677814,0.0,-7103.755904,0.0,-6996.346363
3,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10640.11,...,0.0,-6906.926101,0.0,-6803.167560,0.0,-6869.029043,0.0,-6775.713318,0.0,-6710.266359
4,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,8883.71,...,0.0,-6763.564576,0.0,-6729.711785,0.0,-6477.043476,0.0,-6428.873663,0.0,-6235.439768
5,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,10417.52,...,0.0,-6751.511495,0.0,-6381.049288,0.0,-6240.265253,0.0,-6339.782894,0.0,-5974.932704
6,100,0.05,0.01,100.0,106.0,114.49,125.97,141.16,161.05,9121.59,...,0.0,-6279.246450,0.0,-5755.883353,0.0,-5903.336973,0.0,-5152.134491,0.0,-5240.098605
7,100,0.05,0.05,100.0,110.0,132.25,172.80,244.14,371.29,9112.21,...,0.0,-7695.265680,0.0,-7695.265680,0.0,-7695.265680,0.0,-7695.265680,0.0,-7695.265680
8,100,0.05,0.05,100.0,110.0,132.25,172.80,244.14,371.29,11331.42,...,0.0,-7401.485095,0.0,-7440.200572,0.0,-7301.451956,0.0,-7045.881979,0.0,-6917.820641
9,100,0.05,0.05,100.0,110.0,132.25,172.80,244.14,371.29,9362.13,...,0.0,-7178.747543,0.0,-7147.844249,0.0,-6787.569766,0.0,-6516.758062,0.0,-5985.566839
